# Pre-Trained Word embeedings




In [ ]:
# RUN ONCE load models + data
#!python -m spacy download en_core_web_lg
#!python -m spacy download en_core_web_md
#import nltk
#nltk.download('stopwords')

# get data via wget if it doesn't exist
#!wget -nc https://bitbucket.org/omerlevy/hyperwords/raw/f5a01ea3e44c37c40dcb8d7b813ba3baf330eac0/testsets/analogy/msr.txt
#!wget -nc https://bitbucket.org/omerlevy/hyperwords/raw/f5a01ea3e44c37c40dcb8d7b813ba3baf330eac0/testsets/ws/ws353.txt
#!wget -nc https://bitbucket.org/omerlevy/hyperwords/raw/f5a01ea3e44c37c40dcb8d7b813ba3baf330eac0/testsets/ws/ws353_relatedness.txt
#!wget -nc https://bitbucket.org/omerlevy/hyperwords/raw/f5a01ea3e44c37c40dcb8d7b813ba3baf330eac0/testsets/ws/ws353_similarity.txt
!wget -nc http://öä.eu/sst5.data.txt
# other option: load data via gdrive mounting
# drive.mount('/gdrive')
# %cd /gdrive/My Drive/sst5/<

--2020-04-20 07:28:59--  http://xn--4ca9a.eu/sst5.data.txt
Resolving xn--4ca9a.eu (xn--4ca9a.eu)... 37.120.161.172
Connecting to xn--4ca9a.eu (xn--4ca9a.eu)|37.120.161.172|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://xn--4ca9a.eu/sst5.data.txt [following]
--2020-04-20 07:28:59--  https://xn--4ca9a.eu/sst5.data.txt
Connecting to xn--4ca9a.eu (xn--4ca9a.eu)|37.120.161.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1337522 (1,3M) [text/plain]
Saving to: ‘sst5.data.txt’

sst5.data.txt       100%[===================>]   1,28M  4,17MB/s    in 0,3s    

2020-04-20 07:29:00 (4,17 MB/s) - ‘sst5.data.txt’ saved [1337522/1337522]



In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import numpy as np
import spacy 
import sklearn
from collections import Counter, defaultdict
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import spearmanr
mapl = lambda x, f: list(map(x, f))
mapa = lambda x, f: np.array(list(map(x, f)))
flatten = lambda l: [item for sublist in l for item in sublist]

In [ ]:
msr = pd.read_csv('msr.txt',delimiter='\t',header=None); msr

,0,1,2,3
0,good,better,rough,rougher
1,better,good,rougher,rough
2,good,best,rough,roughest
3,best,good,roughest,rough
4,best,better,roughest,rougher
...,...,...,...,...
7995,sent,send,avoided,avoid
7996,send,sends,avoid,avoids
7997,sends,send,avoids,avoid
7998,sends,sent,avoids,avoided


In [ ]:
w353_similarity_data = pd.read_csv('ws353_similarity.txt',delimiter= '\t',header=None); w353_similarity_data

,0,1,2
0,tiger,cat,7.35
1,tiger,tiger,10.00
2,plane,car,5.77
3,train,car,6.31
4,television,radio,6.77
...,...,...,...
198,rooster,voyage,0.62
199,noon,string,0.54
200,chord,smile,0.54
201,professor,cucumber,0.31


In [ ]:
w353_relatedness_data =  pd.read_csv('ws353_relatedness.txt',delimiter='\t',header=None); w353_relatedness_data

,0,1,2
0,computer,keyboard,7.62
1,Jerusalem,Israel,8.46
2,planet,galaxy,8.11
3,canyon,landscape,7.53
4,OPEC,country,5.63
...,...,...,...
247,rooster,voyage,0.62
248,noon,string,0.54
249,chord,smile,0.54
250,professor,cucumber,0.31


# Word Embedding Evaluation

### Pre-trained Word Embeddings

#### word2vec

In [ ]:
nlp = spacy.load("en_core_web_md")  #load in the model 
print(*nlp.pipeline, sep = "\n")

def tok_extract(line): 
    return [tok.vector for tok in line]

def to_line(df):
    # spacy tokenizes 's as a separate token -> the dimensions don't match up, so we have to remove it
    # also it expects a space-separatated text, so we feed in the 8000x4 as 8000 lines of 4 space separated words
    return [" ".join(line).replace("'", "") for line in df.to_numpy()]

('tagger', <spacy.pipeline.pipes.Tagger object at 0x1a1b01e890>)
('parser', <spacy.pipeline.pipes.DependencyParser object at 0x1a1b141050>)
('ner', <spacy.pipeline.pipes.EntityRecognizer object at 0x1a1b1410c0>)


In [ ]:
# msr
docl_msr_w = nlp.pipe(to_line(msr))
w2v_msr = np.array(mapl(tok_extract, docl_msr_w)); w2v_msr.shape

(8000, 4, 300)

In [ ]:
# w353_relatedness
docl_relat_w = nlp.pipe(to_line(w353_relatedness_data[[0,1]]))
w2v_relatedness = np.array(mapl(tok_extract, docl_relat_w)); w2v_relatedness.shape

(252, 2, 300)

In [ ]:
#w353_similarity
docl_sim_w = nlp.pipe(to_line(w353_similarity_data[[0,1]]))
w2v_similarity = np.array(mapl(tok_extract, docl_sim_w)); w2v_similarity.shape

(203, 2, 300)

In [ ]:
words = "man woman"
tokens = nlp(words) 
  
for token in tokens: 
    # Printing the following attributes of each token. 
    # text: the word string, has_vector: if it contains 
    # a vector representation in the model,  
    # vector_norm: the algebraic norm of the vector, 
    # is_oov: if the word is out of vocabulary. 
    print(token.text, token.has_vector, token.vector_norm, token.is_oov) 
  
print("Similarity:", tokens[0].similarity(tokens[1] )*10)

man True 6.352939 False
woman True 6.8987513 False
Similarity: 7.40174412727356


#### GloVe
The en_core_web_lg model already has GloVe built in
https://spacy.io/usage/vectors-similarity#custom-vectors-coverage

Quote: *For instance, the en_vectors_web_lg model provides 300-dimensional GloVe vectors for over 1 million terms of English.*

In [ ]:
nlp = spacy.load("en_core_web_lg")  #load in the model 
print(*nlp.pipeline, sep = "\n")

('tagger', <spacy.pipeline.pipes.Tagger object at 0x1a1af0b810>)
('parser', <spacy.pipeline.pipes.DependencyParser object at 0x1a51513e50>)
('ner', <spacy.pipeline.pipes.EntityRecognizer object at 0x1a51513b40>)


In [ ]:
#msr
docl_msr_glov = nlp.pipe(to_line(msr))
glov_msr = np.array(mapl(tok_extract, docl_msr_glov)); glov_msr.shape

(8000, 4, 300)

In [ ]:
# w353_relatedness, only take [0,1] as [2] is the scores
docl_relat_glov = nlp.pipe(to_line(w353_relatedness_data[[0,1]]))
glov_relatedness = np.array(mapl(tok_extract, docl_relat_glov)); glov_relatedness.shape

(252, 2, 300)

In [ ]:
#w353_similarity, only take [0,1] as [2] is the scores
docl_sim_glov = nlp.pipe(to_line(w353_similarity_data[[0,1]]))
glov_similarity = np.array(mapl(tok_extract, docl_sim_glov)); glov_similarity.shape

(203, 2, 300)

In [ ]:
#sanity checker 

words = "man woman"
tokens = nlp(words) 
  
for token in tokens: 
    # Printing the following attributes of each token. 
    # text: the word string, has_vector: if it contains 
    # a vector representation in the model,  
    # vector_norm: the algebraic norm of the vector, 
    # is_oov: if the word is out of vocabulary. 
    print(token.text, token.has_vector, token.vector_norm, token.is_oov) 
  
token1, token2 = tokens[0], tokens[1] 
  
print("Similarity:", token1.similarity(token2)*10)

man True 6.352939 False
woman True 6.8987513 False
Similarity: 7.40174412727356


In [ ]:
# helper functions for similarity and analogy
from numpy.linalg import norm

def cosine(a, b):  # works identically to scipy.spatial.distance.cosine
    return 1.0 - a @ b / (norm(a) * norm(b)) 

def three_cos_add(a1, a2, b1, b2):  # 3CosAdd # b2 needs to be vector in the model all 
    return cosine(b2, a2 - a1 + b1) # implementation from LevyA
    #return cosine (b2, b1) - cosine (b2, a1) + cosine (b2, a2) # implementation from LevyB, chapter 6

def three_cos_mul(a1, a2, b1, b2, epsilon=0.001):  # 3CosMul
    return cosine(b2, b1) * cosine(b2, a2) / (cosine(b2, a1) + epsilon) # implementation from LevyA

### Analogy Task 1

In [ ]:
def analogy(data, method, debug_print=False): #analogy task function
    total_score = 0
    for id_correct, (a1, b1, a2, _) in enumerate(data):
        #a1 = nlp(msr[0][id_correct]).vector
        #b1 = nlp(msr[1][id_correct]).vector
        #a2 = nlp(msr[2][id_correct]).vector
        scores = [0]*len(data)
        for id2, (_, _, _, b2) in enumerate(data):
            #b2 = nlp(msr[3][id_correct]).vector
            score = method(a1, b1, a2, b2)
            scores[id2] = score
        best_scoring = np.argmax(scores)
        
        if debug_print:
            print(f"Correct: {msr[0][id_correct]}:{msr[1][id_correct]} = {msr[2][id_correct]}:{msr[3][id_correct]  } - scores {scores[id_correct]}")
            print(f"Best:    {msr[0][id_correct]}:{msr[1][id_correct]} = {msr[2][id_correct]}:{msr[3][best_scoring]} - scores {scores[best_scoring]}\n")
        if np.argmax(scores) == id_correct:
            total_score+=1
    return total_score/len(data)

#word2vec
for data_name, data in [("w2v", w2v_msr), ("glove", glov_msr)]:
    for method in [three_cos_add, three_cos_mul]:
        score = analogy(data[:16], three_cos_add, debug_print=True)
        print(f"score for {data_name} {method.__name__}: {score}\n\n")

Correct: good:better = rough:rougher - scores 0.465084969997406
Best:    good:better = rough:meanest - scores 0.9205763563513756

Correct: better:good = rougher:rough - scores 0.49284571409225464
Best:    better:good = rougher:meanest - scores 0.9684790633618832

Correct: good:best = rough:roughest - scores 0.48699039220809937
Best:    good:best = rough:meaner - scores 0.8533295392990112

Correct: best:good = roughest:rough - scores 0.47195178270339966
Best:    best:good = roughest:meaner - scores 0.7584288716316223

Correct: best:better = roughest:rougher - scores 0.5762052536010742
Best:    best:better = roughest:meanest - scores 0.7005199790000916

Correct: better:best = rougher:roughest - scores 0.6290794014930725
Best:    better:best = rougher:mean - scores 0.9664024896919727

Correct: good:better = mean:meaner - scores 0.6209504902362823
Best:    good:better = mean:roughest - scores 0.8547815978527069

Correct: better:good = meaner:mean - scores 0.7780361622571945
Best:    better

Even though the paper [*Improving Distributional Similaritywith Lessons Learned from Word Embeddings* (LevyA)](https://www.aclweb.org/anthology/Q15-1016.pdf) describes the formulas for 3CosAdd and references the paper [*Linguistic regularities in sparse and explicit word representations* (LevyB)](https://www.aclweb.org/anthology/W14-1618.pdf) and we checked with [an implementation of the paper on github from hrldcpr](https://github.com/hrldcpr/word-embeddings/blob/master/analogies.py), our implementation seems to perform exceptionally bad on our data.

To show that we did all the steps to this point correctly, we print the intended value and its score, as well as the incorrectly best scornig values. We double-checked it by passing every [a1, a2, b1, b2] value of the original msr dataset through nlp(word).vector to verify that nothing went wrong inbetween. It seems that words like there is a bias for mean/meaner in the first 16 words, as they get picked overproportionally.

### Similarity Tasks

In [ ]:
#similarity task 1
#word2vec

def similarity(data,score):
    results = []
    i=0
    for x, y in data:
        cos_sim = np.dot(x,y) /(norm(x)*norm(y))
        results.append((cos_sim,score[i]))
        i+=1
    actual,expected =zip(*results)  
    return spearmanr(actual, expected)

for test_name, d0, d1 in [
    ("w2v similarity  ", w2v_similarity, w353_similarity_data[2]), 
    ("glove similarity", glov_similarity,w353_similarity_data[2]),
    ("w2v relatedness ", w2v_relatedness,w353_relatedness_data[2]),
    ("glov relatedness", glov_relatedness,w353_relatedness_data[2])]:
    score = similarity(d0, d1)
    print(f"{test_name}\tcorrelation: {score.correlation}\tpvalue: {score.pvalue}")

w2v similarity  	correlation: 0.7773731113888416	pvalue: 2.4665099782450488e-42
glove similarity	correlation: 0.8015441627816334	pvalue: 8.958292823931727e-47
w2v relatedness 	correlation: 0.6435221232854792	pvalue: 7.371981209320379e-31
glov relatedness	correlation: 0.644302760922754	pvalue: 5.940670174531038e-31


In this task we took the results of embeededings which we got by running our data through the spacy models and extracting the embeedings which we get by the two methods(Word2vec and GloVe) and ran a similarity evaluation of the embeedings to see how the two embeedings differ from each other.

For the expirement we ran a simple cosine similarity test which told us how similar word1 is to word2 and compared it to the human score. 

The result of our expirement is that we see that GloVe achives a higher correlation, however that is also due to the model which uses GloVe uses a larger corpus. As mentioned in the paper a larger corpus would allow for higher scores to be achieved.

##Document Classification

In [ ]:
df = pd.read_csv('sst5.data.txt'); df

,sentence_id,text,label
0,0,Reno himself can take credit for most of the m...,1
1,1,"Despite the film 's shortcomings , the stories...",1
2,2,"Despite its dry wit and compassion , the film ...",-1
3,3,The central character is n't complex enough to...,0
4,4,Rifkin no doubt fancies himself something of a...,-2
...,...,...,...
11839,11839,"Just as moving , uplifting and funny as ever .",1
11840,11840,Davis ... is so enamored of her own creation t...,-1
11841,11841,"An exhilarating futuristic thriller-noir , Min...",2
11842,11842,I got a headache watching this meaningless dow...,-2


In [ ]:
nlp = spacy.load("en_core_web_md")
X = np.array(df["text"])
Y = np.array(df["label"])

# pass the whole X through the NLP, even if it's just for stemming + tokenization
docl = list(nlp.pipe(X)) # returns just an iterator -> make list

In [ ]:
# as everything above here takes forever, cell split here so we never have to recalculate docl
stem_doc = mapl(lambda line: [tok.lemma_ for tok in line], docl)

# split in train and test
text_train, text_test, label_train, label_test = train_test_split(docl,Y,test_size = 0.2,random_state = 42)

# generate a vocabulary just from text_train
vocab = sorted(set(flatten(stem_doc)))
vocab_opt = {k:v for k,v in enumerate(vocab)} # dicts are optimized for lookup

In [ ]:
def doc2vec(tokens):
    return np.mean([tok.vector for tok in tokens], axis=0)

text_train_vec = mapa(doc2vec, text_train)
text_test_vec = mapa(doc2vec, text_test)
text_train_vec.shape

(9475, 300)

*Incorporating IDF. In this experiment, we extend the method to create document vectors by also
taking into account the Inverse Document Frequency (IDF) of terms. To do it, calculate and store
the IDF values of words given the collection of the classification task. The document embedding
is then defined as follows:
<br><br>
vd = weighted average of vt_i with weight idf(ti)*

In [ ]:
docf = Counter(flatten(stem_doc)) # count of all stemmed tokens
docf = defaultdict(lambda: np.inf, docf) # to ignore OOB words -> defaultscore = inf -> idf = 0

def doc2idfvec(tokens): # if a token appears N times, it will be added N times
    return np.mean([tok.vector / docf[tok.text] for tok in tokens], axis=0)

text_train_idfvec = mapa(doc2idfvec, text_train)
text_test_idfvec = mapa(doc2idfvec, text_test)
text_train_idfvec.shape

(9475, 300)

In [ ]:
print("Dtype\tClassifier\ttrain_pred\ttest_pred")
for data_name, train, test in [["AVG", text_train_vec, text_test_vec], ["IDF", text_train_idfvec, text_test_idfvec]]:
    for classifier in [RandomForestClassifier(random_state=42, n_estimators=100),
                      LinearSVC()]:
        fit = classifier.fit(train, label_train)
        
        pred_train = classifier.predict(train)
        acc_train = accuracy_score(label_train, pred_train)
        
        pred_test = classifier.predict(test)
        acc_test = accuracy_score(label_test, pred_test)
        
        cname = str(classifier).split("(")[0][:12]
        print(data_name, cname, "{:.7f}".format(acc_train), "{:.7f}".format(acc_test), sep="\t")

Dtype	Classifier	train_pred	test_pred
AVG	RandomForest	1.0000000	0.3862389
AVG	LinearSVC	0.4909763	0.4221190
IDF	RandomForest	0.9977836	0.3195441
IDF	LinearSVC	0.3856464	0.3326298




 TF-IDF (and therefore also idf) performs significantly worse in the validation (test_pred) - this is why most libs like spacy internally use plain averages (as we showed in our Assignment1)

Also, random forest massively overfits for cases like this (accuracy on train data is 99-100%), and predictors that take the geometry and proximity of the embedding space into account like SVMs or KNN perform rather good.